# exp057_opt

In [7]:
import os
import sys
import gc
import itertools
import pickle
import pathlib
import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv('UTILS_PATH'))

import pandas as pd
import numpy as np
import cudf
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import matplotlib.pyplot as plt
import seaborn as sns

import line_notify

In [8]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

ディレクトリ設定

In [9]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
#exp_name = os.path.dirname(__file__).split('/')[-1]
exp_name = 'exp057'
os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

データ読み込み

In [10]:
ml_train = pd.read_csv(OUTPUT_DIR + 'exp055/ml_train.csv')
query_train = ml_train['customer_id'].value_counts().sort_index()
ml_train = ml_train.sort_values('customer_id').reset_index(drop=True)

ml_valid = pd.read_csv(OUTPUT_DIR + 'exp055/ml_valid.csv')
query_valid = ml_valid['customer_id'].value_counts().sort_index()
ml_valid = ml_valid.sort_values('customer_id').reset_index(drop=True)

In [11]:
def apk(y_true, y_pred, K=12):
    assert(len(y_true) == len(y_pred))
    apks = []
    for idx in range(len(y_true)):
        y_i_true = y_true[idx]
        y_i_pred = y_pred[idx]

        # 予測値の数と重複の確認
        assert(len(y_i_pred) <= K)
        assert(len(np.unique(y_i_pred)) == len(y_i_pred))

        sum_precision = 0.0
        num_hits = 0.0

        for i, p in enumerate(y_i_pred):
            if p in y_i_true:
                num_hits += 1
                precision = num_hits / (i+1)
                sum_precision += precision
        apk = sum_precision / min(len(y_i_true), K)
        apks.append(apk)
    return apks

In [12]:
# 学習データの作成
# 1週ずつローリングして学習データを生成
train_start = '2020-09-09'
valid_start = '2020-09-16'
valid_end = '2020-09-22'

In [13]:
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'learning_rate': 0.01,
    'ndcg_eval_at': [12],
    'n_estimators': 10000,
    'boosting_type': 'gbdt',
    'random_state': 42
}

In [14]:
# model生成
target = 'buy'
not_use_cols = ['customer_id', 'article_id', target]
features = [c for c in ml_train.columns if c not in not_use_cols]
fi = pd.DataFrame()
vl_pred = np.zeros(len(ml_valid))
scores = []
# 学習
tr_x, tr_y = ml_train[features], ml_train[target]
vl_x, vl_y = ml_valid[features], ml_valid[target]
tr_data = lgb.Dataset(tr_x, label=tr_y, group=query_train)
vl_data = lgb.Dataset(vl_x, label=vl_y, group=query_valid)


model = optuna_lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                num_boost_round=20000, early_stopping_rounds=100,verbose_eval=1000)

[I 2022-04-15 21:20:42,114] A new study created in memory with name: no-name-347841fb-3898-4e42-8e42-cddf48233597
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via '

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.219078 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23111
[LightGBM] [Info] Number of data points in the train set: 1047500, number of used features: 130
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's ndcg@12: 0.978329	valid_1's ndcg@12: 0.802583


feature_fraction, val_score: 0.802651:  14%|#4        | 1/7 [03:03<18:20, 183.43s/it]

Early stopping, best iteration is:
[940]	valid_0's ndcg@12: 0.97812	valid_1's ndcg@12: 0.802651


/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

Early stopping, best iteration is:
[490]	valid_0's ndcg@12: 0.984387	valid_1's ndcg@12: 0.80321


/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.166036 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23111
[LightGBM] [Info] Number of data points in the train set: 1047500, number of used features: 130
Training until validation scores don't improve for 100 rounds


min_data_in_leaf, val_score: 0.803801: 100%|##########| 5/5 [20:35<00:00, 247.04s/it]

Early stopping, best iteration is:
[713]	valid_0's ndcg@12: 0.986398	valid_1's ndcg@12: 0.803471


In [15]:
model.params

{'objective': 'lambdarank',
 'metric': 'ndcg',
 'learning_rate': 0.01,
 'ndcg_eval_at': [12],
 'boosting_type': 'gbdt',
 'random_state': 42,
 'feature_pre_filter': False,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 214,
 'feature_fraction': 0.9159999999999999,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 100}

In [16]:
model.best_score

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('ndcg@12', 0.9873358682730662)]),
             'valid_1': OrderedDict([('ndcg@12', 0.8038012399705796)])})